In [7]:
# first exp
import os
import random
import numpy as np
import time

import torch
import torch.optim as optim
from torchvision import transforms
import torch.optim.lr_scheduler as lr_scheduler

from model_train import CNN, CNNwithSEBlock, CNN3D, CNNwithSEBlock3D, UNet, UNetwithSEBlock, UNetwithSelfattention, UNet3D, UNetwithSEBlock3D, UNetwithSelfattention3D
from DataSet import MaxMinNormalizeGlobalPerChannel,MyDataSet, dataset_2
from train_and_eval import train_one_epoch, evaluate, WeightedMSELoss

random.seed(26)
np.random.seed(26)
torch.manual_seed(26)
torch.cuda.manual_seed(26)
torch.cuda.manual_seed_all(26) 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"  # 或者 ":4096:8"

model_dict = {
    'CNN': CNN,
    'CNNwithSEBlock': CNNwithSEBlock,
    'CNN3D': CNN3D,
    'CNNwithSEBlock3D': CNNwithSEBlock3D,
    'UNet': UNet,
    'UNetwithSEBlock': UNetwithSEBlock,
    'UNetwithSelfattention': UNetwithSelfattention,
    'UNet3D': UNet3D,
    'UNetwithSEBlock3D': UNetwithSEBlock3D,
    'UNetwithSelfattention3D': UNetwithSelfattention3D,
}

class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        """
        :param patience: 如果在多少个epoch内验证集损失没有改善，则提前停止训练
        :param delta: 在认为损失有改善时，损失变化的最小值
        """
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.best_epoch = 0
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_epoch = epoch
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0  # 重置计数器
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Early stopping at epoch {epoch + 1} (no improvement in validation loss for {self.patience} epochs).")
                self.early_stop = True

# 在每次训练之前根据模型名实例化模型
def get_model(model_name):
    return model_dict[model_name]()

def train(model_name, testloader, valloader, epochs, device, earlystoplimit, lr):
    model = get_model(model_name).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    loss_function = WeightedMSELoss()
    early_stopping = EarlyStopping(patience=20, delta=earlystoplimit)
    start_time = time.time()

    best_model = model
    best_val_loss = 10000
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, optimizer, testloader, device, epoch, loss_function)
        scheduler.step()
        val_loss = evaluate(model, valloader, device, loss_function)
        
        # 输出每个epoch的损失
        print(f" Epoch {epoch + 1}: Train Loss: {train_loss:.6f}, Validation Loss: {val_loss:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if epoch > 50 :#设置模型保存间隔
                best_model = model
        early_stopping(val_loss, epoch)
        if early_stopping.early_stop:
            break
    torch.save(best_model.state_dict(), f"/home/linux/3.3lab/outcomes/01/{model_name}.pth")
    training_time = time.time() - start_time
    return {
        'model_name': model_name,
        'model_loss': best_val_loss,
        'training_time': training_time,
    }


def main(args):

    data_transform = {
        "without_jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()]),
        "jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()])}
    # 实例化训练数据集
    data_set = MyDataSet(img_dir=args.img_dir,
                        group_size=10000,
                        size_in = 10000,
                        splition = True,
                        split_shuffle = False,
                        transform=data_transform["without_jet"])
    train_dataset = dataset_2(data_set.train_X, data_set.train_Y)
    val_dataset = dataset_2(data_set.val_X, data_set.val_Y)
    test_dataset = dataset_2(data_set.test_X, data_set.test_Y)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=200, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=200, shuffle=False)
    print(len(train_dataset))
    print(len(test_dataset))
    
    all_results = []
    # 训练每个模型并记录结果
    for model_name in model_dict.keys():
        result = train(model_name, train_dataloader, val_dataloader, epochs=args.epochs,
                                        device=args.device, earlystoplimit=args.earlystoplimit, lr=args.lr)
        all_results.append(result)

    # 输出所有模型的结果
    for result in all_results:
        print(f"Model: {result['model_name']}")
        print(f"Validation Loss: {result['model_loss']}")
        print(f"Training Time: {result['training_time']}")
        print("-" * 50)

class Args:
    def __init__(self):
        self.epochs = 1000
        self.batch_size = 200
        self.lr = 0.001
        self.img_dir = '/home/linux/3.3lab/Gauss_S1.00_NL0.30_B0.50_Jet/Gauss_S1.00_NL0.30_B0.50/Gauss_S1.00_NL0.30_B0.50' 
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.earlystoplimit = 0


opt = Args()

if __name__ == "__main__":
    main(opt)

transformation is not None
8000
1000
 Epoch 1: Train Loss: 0.020664, Validation Loss: 0.029594
 Epoch 2: Train Loss: 0.003047, Validation Loss: 0.003182
 Epoch 3: Train Loss: 0.001342, Validation Loss: 0.001283
 Epoch 4: Train Loss: 0.000975, Validation Loss: 0.000897
 Epoch 5: Train Loss: 0.000837, Validation Loss: 0.000789
 Epoch 6: Train Loss: 0.000753, Validation Loss: 0.000721
 Epoch 7: Train Loss: 0.000693, Validation Loss: 0.000671
 Epoch 8: Train Loss: 0.000649, Validation Loss: 0.000629
 Epoch 9: Train Loss: 0.000612, Validation Loss: 0.000600
 Epoch 10: Train Loss: 0.000583, Validation Loss: 0.000575
 Epoch 11: Train Loss: 0.000557, Validation Loss: 0.000554
 Epoch 12: Train Loss: 0.000537, Validation Loss: 0.000534
 Epoch 13: Train Loss: 0.000519, Validation Loss: 0.000526
 Epoch 14: Train Loss: 0.000506, Validation Loss: 0.000506
 Epoch 15: Train Loss: 0.000490, Validation Loss: 0.000493
 Epoch 16: Train Loss: 0.000477, Validation Loss: 0.000480
 Epoch 17: Train Loss: 0.000

In [13]:
# 2ed exp
import os
import random
import numpy as np
import time

import torch
import torch.optim as optim
from torchvision import transforms
import torch.optim.lr_scheduler as lr_scheduler

from model_train import CNN, CNNwithSEBlock, CNN3D, CNNwithSEBlock3D, UNet, UNetwithSEBlock, UNetwithSelfattention, UNet3D, UNetwithSEBlock3D, UNetwithSelfattention3D
from DataSet import MaxMinNormalizeGlobalPerChannel,MyDataSet, dataset_2
from train_and_eval import train_one_epoch, evaluate, WeightedMSELoss

random.seed(26)
np.random.seed(26)
torch.manual_seed(26)
torch.cuda.manual_seed(26)
torch.cuda.manual_seed_all(26) 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"  # 或者 ":4096:8"

model_dict = {
    'CNN': CNN,
    'CNNwithSEBlock': CNNwithSEBlock,
    'CNN3D': CNN3D,
    'CNNwithSEBlock3D': CNNwithSEBlock3D,
    'UNet': UNet,
    'UNetwithSEBlock': UNetwithSEBlock,
    'UNetwithSelfattention': UNetwithSelfattention,
    'UNet3D': UNet3D,
    'UNetwithSEBlock3D': UNetwithSEBlock3D,
    'UNetwithSelfattention3D': UNetwithSelfattention3D,
}

class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        """
        :param patience: 如果在多少个epoch内验证集损失没有改善，则提前停止训练
        :param delta: 在认为损失有改善时，损失变化的最小值
        """
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.best_epoch = 0
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_epoch = epoch
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0  # 重置计数器
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Early stopping at epoch {epoch + 1} (no improvement in validation loss for {self.patience} epochs).")
                self.early_stop = True

# 在每次训练之前根据模型名实例化模型
def get_model(model_name):
    return model_dict[model_name]()

def train(model_name, testloader, valloader, epochs, device, earlystoplimit, lr):
    model = get_model(model_name).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    loss_function = WeightedMSELoss()
    early_stopping = EarlyStopping(patience=10, delta=earlystoplimit)
    start_time = time.time()

    best_model = model
    best_val_loss = 10000
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, optimizer, testloader, device, epoch, loss_function)
        scheduler.step()
        val_loss = evaluate(model, valloader, device, loss_function)
        
        # 输出每个epoch的损失
        print(f" Epoch {epoch + 1}: Train Loss: {train_loss:.6f}, Validation Loss: {val_loss:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if epoch > 20 :#设置模型保存间隔
                best_model = model
        early_stopping(val_loss, epoch)
        if early_stopping.early_stop:
            break
    torch.save(best_model.state_dict(), f"/home/linux/3.3lab/outcomes/02/{model_name}.pth")
    training_time = time.time() - start_time
    return {
        'model_name': model_name,
        'model_loss': best_val_loss,
        'training_time': training_time,
    }


def main(args):

    data_transform = {
        "without_jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()]),
        "jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()])}
    # 实例化训练数据集
    data_set = MyDataSet(img_dir=args.img_dir,
                        group_size=10000,
                        size_in = 10000,
                        splition = True,
                        split_shuffle = False,
                        transform=data_transform["without_jet"])
    train_dataset = dataset_2(data_set.train_X, data_set.train_Y)
    val_dataset = dataset_2(data_set.val_X, data_set.val_Y)
    test_dataset = dataset_2(data_set.test_X, data_set.test_Y)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=200, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=200, shuffle=False)
    print(len(train_dataset))
    print(len(test_dataset))
    
    all_results = []
    # 训练每个模型并记录结果
    for model_name in model_dict.keys():
        result = train(model_name, train_dataloader, val_dataloader, epochs=args.epochs,
                                        device=args.device, earlystoplimit=args.earlystoplimit, lr=args.lr)
        all_results.append(result)

    # 输出所有模型的结果
    for result in all_results:
        print(f"Model: {result['model_name']}")
        print(f"Validation Loss: {result['model_loss']}")
        print(f"Training Time: {result['training_time']}")
        print("-" * 50)

class Args:
    def __init__(self):
        self.epochs = 1000
        self.batch_size = 200
        self.lr = 0.001
        self.img_dir = '/home/linux/3.3lab/Gauss_S1.00_NL0.30_B0.50/Gauss_S1.00_NL0.30_B0.50' 
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.earlystoplimit = 1e-7


opt = Args()

if __name__ == "__main__":
    main(opt)


transformation is not None
8000
1000
 Epoch 1: Train Loss: 0.020664, Validation Loss: 0.029594
 Epoch 2: Train Loss: 0.003047, Validation Loss: 0.003182
 Epoch 3: Train Loss: 0.001342, Validation Loss: 0.001283
 Epoch 4: Train Loss: 0.000975, Validation Loss: 0.000897
 Epoch 5: Train Loss: 0.000837, Validation Loss: 0.000789
 Epoch 6: Train Loss: 0.000753, Validation Loss: 0.000721
 Epoch 7: Train Loss: 0.000693, Validation Loss: 0.000671
 Epoch 8: Train Loss: 0.000649, Validation Loss: 0.000629
 Epoch 9: Train Loss: 0.000612, Validation Loss: 0.000600
 Epoch 10: Train Loss: 0.000583, Validation Loss: 0.000575
 Epoch 11: Train Loss: 0.000557, Validation Loss: 0.000554
 Epoch 12: Train Loss: 0.000537, Validation Loss: 0.000534
 Epoch 13: Train Loss: 0.000519, Validation Loss: 0.000526
 Epoch 14: Train Loss: 0.000506, Validation Loss: 0.000506
 Epoch 15: Train Loss: 0.000490, Validation Loss: 0.000493
 Epoch 16: Train Loss: 0.000477, Validation Loss: 0.000480
 Epoch 17: Train Loss: 0.000

In [7]:
# about new attention block
import os
import random
import numpy as np
import time

import torch
import torch.optim as optim
from torchvision import transforms
import torch.optim.lr_scheduler as lr_scheduler

# from model_train import CNN, CNNwithSEBlock, CNN3D, CNNwithSEBlock3D, UNet, UNetwithSEBlock, UNetwithSelfattention, UNet3D, UNetwithSEBlock3D, UNetwithSelfattention3D
from model_new import ResNet,CNN_with_Attention
from DataSet import MaxMinNormalizeGlobalPerChannel,MyDataSet, dataset_2
from train_and_eval import train_one_epoch, evaluate, WeightedMSELoss

random.seed(26)
np.random.seed(26)
torch.manual_seed(26)
torch.cuda.manual_seed(26)
torch.cuda.manual_seed_all(26) 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"  # 或者 ":4096:8"

model_dict = {
    # 'CNN': CNN,
    'CNN_with_Attention': CNN_with_Attention,
    # 'ResNet': ResNet,
    # 'CNNwithSEBlock': CNNwithSEBlock,
    # 'CNN3D': CNN3D,
    # 'CNNwithSEBlock3D': CNNwithSEBlock3D,
    # 'UNet': UNet,
    # 'UNetwithSEBlock': UNetwithSEBlock,
    # 'UNetwithSelfattention': UNetwithSelfattention,
    # 'UNet3D': UNet3D,
    # 'UNetwithSEBlock3D': UNetwithSEBlock3D,
    # 'UNetwithSelfattention3D': UNetwithSelfattention3D,
}

class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        """
        :param patience: 如果在多少个epoch内验证集损失没有改善，则提前停止训练
        :param delta: 在认为损失有改善时，损失变化的最小值
        """
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.best_epoch = 0
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_epoch = epoch
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0  # 重置计数器
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Early stopping at epoch {epoch + 1} (no improvement in validation loss for {self.patience} epochs).")
                self.early_stop = True

# 在每次训练之前根据模型名实例化模型
def get_model(model_name):
    return model_dict[model_name]()

def train(model_name, testloader, valloader, epochs, device, earlystoplimit, lr):
    model = get_model(model_name).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    loss_function = WeightedMSELoss()
    early_stopping = EarlyStopping(patience=10, delta=earlystoplimit)
    start_time = time.time()

    best_model = model
    best_val_loss = 10000
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, optimizer, testloader, device, epoch, loss_function)
        scheduler.step()
        val_loss = evaluate(model, valloader, device, loss_function)
        
        # 输出每个epoch的损失
        print(f" Epoch {epoch + 1}: Train Loss: {train_loss:.6f}, Validation Loss: {val_loss:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if epoch > 20 :#设置模型保存间隔
                best_model = model
        early_stopping(val_loss, epoch)
        if early_stopping.early_stop:
            break
    torch.save(best_model.state_dict(), f"/home/linux/3.3lab/outcomes/04/{model_name}.pth")
    training_time = time.time() - start_time
    return {
        'model_name': model_name,
        'model_loss': best_val_loss,
        'training_time': training_time,
    }




def main(args):

    data_transform = {
        "without_jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()]),
        "jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()])}
    # 实例化训练数据集
    data_set = MyDataSet(img_dir=args.img_dir,
                        group_size=10000,
                        size_in = 10000,
                        splition = True,
                        split_shuffle = False,
                        transform=data_transform["without_jet"])
    train_dataset = dataset_2(data_set.train_X, data_set.train_Y)
    val_dataset = dataset_2(data_set.val_X, data_set.val_Y)
    test_dataset = dataset_2(data_set.test_X, data_set.test_Y)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=200, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=200, shuffle=False)
    print(len(train_dataset))
    print(len(test_dataset))
    
    all_results = []
    # 训练每个模型并记录结果
    for model_name in model_dict.keys():
        result = train(model_name, train_dataloader, val_dataloader, epochs=args.epochs,
                                        device=args.device, earlystoplimit=args.earlystoplimit, lr=args.lr)
        all_results.append(result)

    # 输出所有模型的结果
    for result in all_results:
        print(f"Model: {result['model_name']}")
        print(f"Validation Loss: {result['model_loss']}")
        print(f"Training Time: {result['training_time']}")
        print("-" * 50)

class Args:
    def __init__(self):
        self.epochs = 1000
        self.batch_size = 200
        self.lr = 0.001
        self.img_dir = '/home/linux/3.3lab/Gauss_S1.00_NL0.30_B0.50/Gauss_S1.00_NL0.30_B0.50' 
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.earlystoplimit = 1e-7




opt = Args()


if __name__ == "__main__":
    main(opt)


transformation is not None
8000


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [2]:
import os
import random
import numpy as np
import time

import torch
import torch.optim as optim
from torchvision import transforms
import torch.optim.lr_scheduler as lr_scheduler

from model_CNN import CNN_1, CNNwithSEBlock_1, CNN3D_1, CNNwithSEBlock3D_1,CNN_2, CNNwithSEBlock_2, CNN3D_2, CNNwithSEBlock3D_2,CNN_3, CNNwithSEBlock_3, CNN3D_3, CNNwithSEBlock3D_3,CNN_4, CNNwithSEBlock_4, CNN3D_4, CNNwithSEBlock3D_4
from DataSet import MaxMinNormalizeGlobalPerChannel,MyDataSet, dataset_2
from train_and_eval import train_one_epoch, evaluate, WeightedMSELoss

random.seed(26)
np.random.seed(26)
torch.manual_seed(26)
torch.cuda.manual_seed(26)
torch.cuda.manual_seed_all(26) 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"  # 或者 ":4096:8"

model_dict = {
    # 'CNN_1': CNN_1,
    # 'CNNwithSEBlock_1': CNNwithSEBlock_1,
    # 'CNN3D_1': CNN3D_1,
    # 'CNNwithSEBlock3D_1': CNNwithSEBlock3D_1,
    # 'CNN_2': CNN_2,
    # 'CNNwithSEBlock_2': CNNwithSEBlock_2,
    # 'CNN3D_2': CNN3D_2,
    # 'CNNwithSEBlock3D_2': CNNwithSEBlock3D_2,
    # 'CNN_3': CNN_3,
    'CNNwithSEBlock_3': CNNwithSEBlock_3,
    'CNN3D_3': CNN3D_3,
    'CNNwithSEBlock3D_3': CNNwithSEBlock3D_3,
    'CNN_4': CNN_4,
    'CNNwithSEBlock_4': CNNwithSEBlock_4,
    'CNN3D_4': CNN3D_4,
    'CNNwithSEBlock3D_4': CNNwithSEBlock3D_4,
}

class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        """
        :param patience: 如果在多少个epoch内验证集损失没有改善，则提前停止训练
        :param delta: 在认为损失有改善时，损失变化的最小值
        """
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.best_epoch = 0
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_epoch = epoch
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0  # 重置计数器
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Early stopping at epoch {epoch + 1} (no improvement in validation loss for {self.patience} epochs).")
                self.early_stop = True

# 在每次训练之前根据模型名实例化模型
def get_model(model_name):
    return model_dict[model_name]()

def train(model_name, testloader, valloader, epochs, device, earlystoplimit, lr):
    model = get_model(model_name).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    loss_function = WeightedMSELoss()
    early_stopping = EarlyStopping(patience=20, delta=earlystoplimit)
    start_time = time.time()

    best_model = model
    best_val_loss = 10000
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, optimizer, testloader, device, epoch, loss_function)
        scheduler.step()
        val_loss = evaluate(model, valloader, device, loss_function)
        
        # 输出每个epoch的损失
        print(f" Epoch {epoch + 1}: Train Loss: {train_loss:.6f}, Validation Loss: {val_loss:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if epoch > 50 :#设置模型保存间隔
                best_model = model
        early_stopping(val_loss, epoch)
        if early_stopping.early_stop:
            break
    torch.save(best_model.state_dict(), f"/home/linux/3.3lab/outcomes/CNN/{model_name}.pth")
    training_time = time.time() - start_time
    return {
        'model_name': model_name,
        'model_loss': best_val_loss,
        'training_time': training_time,
    }


def main(args):

    data_transform = {
        "without_jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()]),
        "jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()])}
    # 实例化训练数据集
    data_set = MyDataSet(img_dir=args.img_dir,
                        group_size=10000,
                        size_in = 10000,
                        splition = True,
                        split_shuffle = False,
                        transform=data_transform['without_jet'])
    train_dataset = dataset_2(data_set.train_X, data_set.train_Y)
    val_dataset = dataset_2(data_set.val_X, data_set.val_Y)
    test_dataset = dataset_2(data_set.test_X, data_set.test_Y)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=200, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=200, shuffle=False)
    print(len(train_dataset))
    print(len(test_dataset))
    
    all_results = []
    # 训练每个模型并记录结果
    for model_name in model_dict.keys():
        result = train(model_name, train_dataloader, val_dataloader, epochs=args.epochs,
                                        device=args.device, earlystoplimit=args.earlystoplimit, lr=args.lr)
        all_results.append(result)

    # 输出所有模型的结果
    for result in all_results:
        print(f"Model: {result['model_name']}")
        print(f"Validation Loss: {result['model_loss']}")
        print(f"Training Time: {result['training_time']}")
        print("-" * 50)

class Args:
    def __init__(self):
        self.epochs = 1000
        self.batch_size = 200
        self.lr = 0.001
        self.img_dir = 'Gauss_S1.00_NL0.30_B0.50/Gauss_S1.00_NL0.30_B0.50' 
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.earlystoplimit = 0


opt = Args()

if __name__ == "__main__":
    main(opt)


transformation is not None
8000
1000
 Epoch 1: Train Loss: 0.020119, Validation Loss: 0.023445
 Epoch 2: Train Loss: 0.002864, Validation Loss: 0.001986
 Epoch 3: Train Loss: 0.001212, Validation Loss: 0.001061
 Epoch 4: Train Loss: 0.000950, Validation Loss: 0.000896
 Epoch 5: Train Loss: 0.000830, Validation Loss: 0.000783
 Epoch 6: Train Loss: 0.000747, Validation Loss: 0.000710
 Epoch 7: Train Loss: 0.000686, Validation Loss: 0.000663
 Epoch 8: Train Loss: 0.000640, Validation Loss: 0.000622
 Epoch 9: Train Loss: 0.000603, Validation Loss: 0.000589
 Epoch 10: Train Loss: 0.000574, Validation Loss: 0.000566
 Epoch 11: Train Loss: 0.000551, Validation Loss: 0.000543
 Epoch 12: Train Loss: 0.000530, Validation Loss: 0.000531
 Epoch 13: Train Loss: 0.000514, Validation Loss: 0.000507
 Epoch 14: Train Loss: 0.000494, Validation Loss: 0.000509
 Epoch 15: Train Loss: 0.000482, Validation Loss: 0.000479
 Epoch 16: Train Loss: 0.000470, Validation Loss: 0.000468
 Epoch 17: Train Loss: 0.000

In [3]:
import os
import random
import numpy as np
import time

import torch
import torch.optim as optim
from torchvision import transforms
import torch.optim.lr_scheduler as lr_scheduler

from model_UNet import UNet_1,UNetwithSEBlock_1,UNetwithSelfattention_1,UNet3D_1,UNetwithSEBlock3D_1,UNetwithSelfattention3D_1,UNet_2,UNetwithSEBlock_2,UNetwithSelfattention_2,UNet3D_2,UNetwithSEBlock3D_2,UNetwithSelfattention3D_2,UNet_3,UNetwithSEBlock_3,UNetwithSelfattention_3,UNet3D_3,UNetwithSEBlock3D_3,UNetwithSelfattention3D_3,UNet_4,UNetwithSEBlock_4,UNetwithSelfattention_4,UNet3D_4,UNetwithSEBlock3D_4,UNetwithSelfattention3D_4,UNet_5,UNetwithSEBlock_5,UNetwithSelfattention_5,UNet3D_5,UNetwithSEBlock3D_5,UNetwithSelfattention3D_5,UNet_6,UNetwithSEBlock_6,UNetwithSelfattention_6,UNet3D_6,UNetwithSelfattention3D_6
# from model_UNet import UNetwithSEBlock3D_6
from DataSet import MaxMinNormalizeGlobalPerChannel,MyDataSet, dataset_2
from train_and_eval import train_one_epoch, evaluate, WeightedMSELoss

random.seed(26)
np.random.seed(26)
torch.manual_seed(26)
torch.cuda.manual_seed(26)
torch.cuda.manual_seed_all(26) 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"  # 或者 ":4096:8"

model_dict = {
    'UNet_1': UNet_1,
    'UNetwithSEBlock_1': UNetwithSEBlock_1,
    'UNetwithSelfattention_1': UNetwithSelfattention_1,
    'UNet3D_1': UNet3D_1,
    'UNetwithSEBlock3D_1': UNetwithSEBlock3D_1,
    'UNetwithSelfattention3D_1': UNetwithSelfattention3D_1,
    'UNet_2': UNet_2,
    'UNetwithSEBlock_2': UNetwithSEBlock_2,
    'UNetwithSelfattention_2': UNetwithSelfattention_2,
    'UNet3D_2': UNet3D_2,
    'UNetwithSEBlock3D_2': UNetwithSEBlock3D_2,
    'UNetwithSelfattention3D_2': UNetwithSelfattention3D_2,
    'UNet_3': UNet_3,
    'UNetwithSEBlock_3': UNetwithSEBlock_3,
    'UNetwithSelfattention_3': UNetwithSelfattention_3,
    'UNet3D_3': UNet3D_3,
    'UNetwithSEBlock3D_3': UNetwithSEBlock3D_3,
    'UNetwithSelfattention3D_3': UNetwithSelfattention3D_3,
    'UNet_4': UNet_4,
    'UNetwithSEBlock_4': UNetwithSEBlock_4,
    'UNetwithSelfattention_4': UNetwithSelfattention_4,
    'UNet3D_4': UNet3D_4,
    'UNetwithSEBlock3D_4': UNetwithSEBlock3D_4,
    'UNetwithSelfattention3D_4': UNetwithSelfattention3D_4,
    'UNet_5': UNet_5,
    'UNetwithSEBlock_5': UNetwithSEBlock_5,
    'UNetwithSelfattention_5': UNetwithSelfattention_5,
    'UNet3D_5': UNet3D_5,
    'UNetwithSEBlock3D_5': UNetwithSEBlock3D_5,
    'UNetwithSelfattention3D_5': UNetwithSelfattention3D_5,
    'UNet_6': UNet_6,
    'UNetwithSEBlock_6': UNetwithSEBlock_6,
    'UNetwithSelfattention_6': UNetwithSelfattention_6,
    'UNet3D_6': UNet3D_6,
    # 'UNetwithSEBlock3D_6': UNetwithSEBlock3D_6,
    'UNetwithSelfattention3D_6': UNetwithSelfattention3D_6,
}

class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        """
        :param patience: 如果在多少个epoch内验证集损失没有改善，则提前停止训练
        :param delta: 在认为损失有改善时，损失变化的最小值
        """
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.best_epoch = 0
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_epoch = epoch
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0  # 重置计数器
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Early stopping at epoch {epoch + 1} (no improvement in validation loss for {self.patience} epochs).")
                self.early_stop = True

# 在每次训练之前根据模型名实例化模型
def get_model(model_name):
    return model_dict[model_name]()

def train(model_name, testloader, valloader, epochs, device, earlystoplimit, lr):
    model = get_model(model_name).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    loss_function = WeightedMSELoss()
    early_stopping = EarlyStopping(patience=20, delta=earlystoplimit)
    start_time = time.time()

    best_model = model
    best_val_loss = 10000
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, optimizer, testloader, device, epoch, loss_function)
        scheduler.step()
        val_loss = evaluate(model, valloader, device, loss_function)
        
        # 输出每个epoch的损失
        print(f" Epoch {epoch + 1}: Train Loss: {train_loss:.6f}, Validation Loss: {val_loss:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if epoch > 50 :#设置模型保存间隔
                best_model = model
        early_stopping(val_loss, epoch)
        if early_stopping.early_stop:
            break
    torch.save(best_model.state_dict(), f"/home/linux/3.3lab/outcomes/CNN/{model_name}.pth")
    training_time = time.time() - start_time
    return {
        'model_name': model_name,
        'model_loss': best_val_loss,
        'training_time': training_time,
    }


def main(args):

    data_transform = {
        "without_jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()]),
        "jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()])}
    # 实例化训练数据集
    data_set = MyDataSet(img_dir=args.img_dir,
                        group_size=10000,
                        size_in = 10000,
                        splition = True,
                        split_shuffle = False,
                        transform=data_transform['without_jet'])
    train_dataset = dataset_2(data_set.train_X, data_set.train_Y)
    val_dataset = dataset_2(data_set.val_X, data_set.val_Y)
    test_dataset = dataset_2(data_set.test_X, data_set.test_Y)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=200, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=200, shuffle=False)
    print(len(train_dataset))
    print(len(test_dataset))
    
    all_results = []
    # 训练每个模型并记录结果
    for model_name in model_dict.keys():
        result = train(model_name, train_dataloader, val_dataloader, epochs=args.epochs,
                                        device=args.device, earlystoplimit=args.earlystoplimit, lr=args.lr)
        all_results.append(result)

    # 输出所有模型的结果
    for result in all_results:
        print(f"Model: {result['model_name']}")
        print(f"Validation Loss: {result['model_loss']}")
        print(f"Training Time: {result['training_time']}")
        print("-" * 50)

class Args:
    def __init__(self):
        self.epochs = 1000
        self.batch_size = 200
        self.lr = 0.001
        self.img_dir = 'Gauss_S1.00_NL0.30_B0.50/Gauss_S1.00_NL0.30_B0.50' 
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.earlystoplimit = 0


opt = Args()

if __name__ == "__main__":
    main(opt)


transformation is not None
8000
1000
 Epoch 1: Train Loss: 0.048260, Validation Loss: 0.009730
 Epoch 2: Train Loss: 0.013208, Validation Loss: 0.007903
 Epoch 3: Train Loss: 0.008348, Validation Loss: 0.005607
 Epoch 4: Train Loss: 0.006828, Validation Loss: 0.004442
 Epoch 5: Train Loss: 0.005813, Validation Loss: 0.003504
 Epoch 6: Train Loss: 0.005070, Validation Loss: 0.002872
 Epoch 7: Train Loss: 0.004547, Validation Loss: 0.002397
 Epoch 8: Train Loss: 0.004172, Validation Loss: 0.002126
 Epoch 9: Train Loss: 0.003901, Validation Loss: 0.001946
 Epoch 10: Train Loss: 0.003689, Validation Loss: 0.001810
 Epoch 11: Train Loss: 0.003524, Validation Loss: 0.001701
 Epoch 12: Train Loss: 0.003386, Validation Loss: 0.001618
 Epoch 13: Train Loss: 0.003269, Validation Loss: 0.001557
 Epoch 14: Train Loss: 0.003170, Validation Loss: 0.001515
 Epoch 15: Train Loss: 0.003077, Validation Loss: 0.001492
 Epoch 16: Train Loss: 0.002993, Validation Loss: 0.001458
 Epoch 17: Train Loss: 0.002

In [1]:
import os
import random
import numpy as np
import time

import torch
import torch.optim as optim
from torchvision import transforms
import torch.optim.lr_scheduler as lr_scheduler
from model_CNN import CNN_1, CNN_2, CNN_3, CNN_4

from DataSet import MaxMinNormalizeGlobalPerChannel,MyDataSet, dataset_2
from train_and_eval import train_one_epoch, evaluate

random.seed(26)
np.random.seed(26)
torch.manual_seed(26)
torch.cuda.manual_seed(26)
torch.cuda.manual_seed_all(26) 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"  # 或者 ":4096:8"


model_dict = {
    'CNN_1': CNN_1,
    'CNN_2': CNN_2,
    'CNN_3': CNN_3,
    'CNN_4': CNN_4,
}


class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        """
        :param patience: 如果在多少个epoch内验证集损失没有改善，则提前停止训练
        :param delta: 在认为损失有改善时，损失变化的最小值
        """
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.best_epoch = 0
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_epoch = epoch
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0  # 重置计数器
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Early stopping at epoch {epoch + 1} (no improvement in validation loss for {self.patience} epochs).")
                self.early_stop = True

# 在每次训练之前根据模型名实例化模型
def get_model(model_name):
    return model_dict[model_name]()

def train(model_name, testloader, valloader, epochs, device, earlystoplimit, lr):
    model = get_model(model_name).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    loss_function = torch.nn.MSELoss()
    early_stopping = EarlyStopping(patience=20, delta=earlystoplimit)
    start_time = time.time()

    best_model = model
    best_val_loss = 10000
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, optimizer, testloader, device, epoch, loss_function)
        scheduler.step()
        val_loss = evaluate(model, valloader, device, loss_function)
        
        # 输出每个epoch的损失
        print(f" Epoch {epoch + 1}: Train Loss: {train_loss:.6f}, Validation Loss: {val_loss:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if epoch > 50 :#设置模型保存间隔
                best_model = model
        early_stopping(val_loss, epoch)
        if early_stopping.early_stop:
            break
    torch.save(best_model.state_dict(), f"/home/linux/3.3lab/outcomes/Old_MSE/{model_name}.pth")
    training_time = time.time() - start_time
    return {
        'model_name': model_name,
        'model_loss': best_val_loss,
        'training_time': training_time,
    }


def main(args):

    data_transform = {
        "without_jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()]),
        "jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()])}
    # 实例化训练数据集
    data_set = MyDataSet(img_dir=args.img_dir,
                        group_size=10000,
                        size_in = 10000,
                        splition = True,
                        split_shuffle = False,
                        transform=data_transform['without_jet'])
    train_dataset = dataset_2(data_set.train_X, data_set.train_Y)
    val_dataset = dataset_2(data_set.val_X, data_set.val_Y)
    test_dataset = dataset_2(data_set.test_X, data_set.test_Y)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=200, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=200, shuffle=False)
    print(len(train_dataset))
    print(len(test_dataset))
    
    all_results = []
    # 训练每个模型并记录结果
    for model_name in model_dict.keys():
        result = train(model_name, train_dataloader, val_dataloader, epochs=args.epochs,
                                        device=args.device, earlystoplimit=args.earlystoplimit, lr=args.lr)
        all_results.append(result)

    # 输出所有模型的结果
    for result in all_results:
        print(f"Model: {result['model_name']}")
        print(f"Validation Loss: {result['model_loss']}")
        print(f"Training Time: {result['training_time']}")
        print("-" * 50)

class Args:
    def __init__(self):
        self.epochs = 1000
        self.batch_size = 200
        self.lr = 0.001
        self.img_dir = 'Gauss_S1.00_NL0.30_B0.50/Gauss_S1.00_NL0.30_B0.50' 
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.earlystoplimit = 0


opt = Args()

if __name__ == "__main__":
    main(opt)


transformation is not None
8000
1000
 Epoch 1: Train Loss: 0.017449, Validation Loss: 0.005693
 Epoch 2: Train Loss: 0.005709, Validation Loss: 0.005692
 Epoch 3: Train Loss: 0.005709, Validation Loss: 0.005691
 Epoch 4: Train Loss: 0.005709, Validation Loss: 0.005690
 Epoch 5: Train Loss: 0.005708, Validation Loss: 0.005689
 Epoch 6: Train Loss: 0.005707, Validation Loss: 0.005687
 Epoch 7: Train Loss: 0.005704, Validation Loss: 0.005684
 Epoch 8: Train Loss: 0.005701, Validation Loss: 0.005678
 Epoch 9: Train Loss: 0.005694, Validation Loss: 0.005667
 Epoch 10: Train Loss: 0.005681, Validation Loss: 0.005634
 Epoch 11: Train Loss: 0.005625, Validation Loss: 0.005124
 Epoch 12: Train Loss: 0.005277, Validation Loss: 0.004839
 Epoch 13: Train Loss: 0.004907, Validation Loss: 0.004739
 Epoch 14: Train Loss: 0.004373, Validation Loss: 0.004535
 Epoch 15: Train Loss: 0.002750, Validation Loss: 0.002471
 Epoch 16: Train Loss: 0.001630, Validation Loss: 0.001496
 Epoch 17: Train Loss: 0.001

In [2]:
import os
import random
import numpy as np
import time

import torch
import torch.optim as optim
from torchvision import transforms
import torch.optim.lr_scheduler as lr_scheduler
from model_train2 import CNN, CNNwithSEBlock, CNN3D, CNNwithSEBlock3D, UNet, UNetwithSEBlock, UNetwithSelfattention, UNet3D, UNetwithSEBlock3D, UNetwithSelfattention3D

from DataSet import MaxMinNormalizeGlobalPerChannel,MyDataSet, dataset_2
from train_and_eval import train_one_epoch, evaluate

random.seed(26)
np.random.seed(26)
torch.manual_seed(26)
torch.cuda.manual_seed(26)
torch.cuda.manual_seed_all(26) 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"  # 或者 ":4096:8"


model_dict = {
    'CNN': CNN,
    'CNNwithSEBlock': CNNwithSEBlock,
    'CNN3D': CNN3D,
    'CNNwithSEBlock3D': CNNwithSEBlock3D,
    'UNet': UNet,
    'UNetwithSEBlock': UNetwithSEBlock,
    'UNetwithSelfattention': UNetwithSelfattention,
    'UNet3D': UNet3D,
    'UNetwithSEBlock3D': UNetwithSEBlock3D,
    'UNetwithSelfattention3D': UNetwithSelfattention3D,
}


class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        """
        :param patience: 如果在多少个epoch内验证集损失没有改善，则提前停止训练
        :param delta: 在认为损失有改善时，损失变化的最小值
        """
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.best_epoch = 0
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_epoch = epoch
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0  # 重置计数器
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Early stopping at epoch {epoch + 1} (no improvement in validation loss for {self.patience} epochs).")
                self.early_stop = True

# 在每次训练之前根据模型名实例化模型
def get_model(model_name):
    return model_dict[model_name]()

def train(model_name, testloader, valloader, epochs, device, earlystoplimit, lr):
    model = get_model(model_name).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    loss_function = torch.nn.MSELoss()
    early_stopping = EarlyStopping(patience=20, delta=earlystoplimit)
    start_time = time.time()

    best_model = model
    best_val_loss = 10000
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, optimizer, testloader, device, epoch, loss_function)
        scheduler.step()
        val_loss = evaluate(model, valloader, device, loss_function)
        
        # 输出每个epoch的损失
        print(f" Epoch {epoch + 1}: Train Loss: {train_loss:.6f}, Validation Loss: {val_loss:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if epoch > 50 :#设置模型保存间隔
                best_model = model
        early_stopping(val_loss, epoch)
        if early_stopping.early_stop:
            break
    torch.save(best_model.state_dict(), f"/home/linux/3.3lab/outcomes/Old_MSE_2/{model_name}.pth")
    training_time = time.time() - start_time
    return {
        'model_name': model_name,
        'model_loss': best_val_loss,
        'training_time': training_time,
    }


def main(args):

    data_transform = {
        "without_jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()]),
        "jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()])}
    # 实例化训练数据集
    data_set = MyDataSet(img_dir=args.img_dir,
                        group_size=10000,
                        size_in = 10000,
                        splition = True,
                        split_shuffle = False,
                        transform=data_transform['without_jet'])
    train_dataset = dataset_2(data_set.train_X, data_set.train_Y)
    val_dataset = dataset_2(data_set.val_X, data_set.val_Y)
    test_dataset = dataset_2(data_set.test_X, data_set.test_Y)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=200, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=200, shuffle=False)
    print(len(train_dataset))
    print(len(test_dataset))
    
    all_results = []
    # 训练每个模型并记录结果
    for model_name in model_dict.keys():
        result = train(model_name, train_dataloader, val_dataloader, epochs=args.epochs,
                                        device=args.device, earlystoplimit=args.earlystoplimit, lr=args.lr)
        all_results.append(result)

    # 输出所有模型的结果
    for result in all_results:
        print(f"Model: {result['model_name']}")
        print(f"Validation Loss: {result['model_loss']}")
        print(f"Training Time: {result['training_time']}")
        print("-" * 50)

class Args:
    def __init__(self):
        self.epochs = 1000
        self.batch_size = 200
        self.lr = 0.001
        self.img_dir = 'Gauss_S1.00_NL0.30_B0.50/Gauss_S1.00_NL0.30_B0.50' 
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.earlystoplimit = 0


opt = Args()

if __name__ == "__main__":
    main(opt)


transformation is not None
8000
1000
 Epoch 1: Train Loss: 0.016758, Validation Loss: 0.005693
 Epoch 2: Train Loss: 0.004611, Validation Loss: 0.004888
 Epoch 3: Train Loss: 0.004371, Validation Loss: 0.004483
 Epoch 4: Train Loss: 0.003857, Validation Loss: 0.003659
 Epoch 5: Train Loss: 0.002448, Validation Loss: 0.002053
 Epoch 6: Train Loss: 0.001607, Validation Loss: 0.001492
 Epoch 7: Train Loss: 0.001181, Validation Loss: 0.001173
 Epoch 8: Train Loss: 0.000899, Validation Loss: 0.000841
 Epoch 9: Train Loss: 0.000706, Validation Loss: 0.000646
 Epoch 10: Train Loss: 0.000577, Validation Loss: 0.000529
 Epoch 11: Train Loss: 0.000496, Validation Loss: 0.000457
 Epoch 12: Train Loss: 0.000419, Validation Loss: 0.000400
 Epoch 13: Train Loss: 0.000377, Validation Loss: 0.000366
 Epoch 14: Train Loss: 0.000350, Validation Loss: 0.000346
 Epoch 15: Train Loss: 0.000332, Validation Loss: 0.000326
 Epoch 16: Train Loss: 0.000313, Validation Loss: 0.000307
 Epoch 17: Train Loss: 0.000

In [2]:
import os
import random
import numpy as np
import time

import torch
import torch.optim as optim
from torchvision import transforms
import torch.optim.lr_scheduler as lr_scheduler
from model_train2 import CNN, CNNwithSEBlock, CNN3D, CNNwithSEBlock3D, UNet, UNetwithSEBlock, UNetwithSelfattention, UNet3D, UNetwithSEBlock3D, UNetwithSelfattention3D

from DataSet import MaxMinNormalizeGlobalPerChannel,MyDataSet, dataset_2
from train_and_eval import train_one_epoch, evaluate,MixedMSE

random.seed(26)
np.random.seed(26)
torch.manual_seed(26)
torch.cuda.manual_seed(26)
torch.cuda.manual_seed_all(26) 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"  # 或者 ":4096:8"


model_dict = {
    'CNN': CNN,
    'CNNwithSEBlock': CNNwithSEBlock,
    'CNN3D': CNN3D,
    'CNNwithSEBlock3D': CNNwithSEBlock3D,
    # 'UNet': UNet,
    # 'UNetwithSEBlock': UNetwithSEBlock,
    # 'UNetwithSelfattention': UNetwithSelfattention,
    # 'UNet3D': UNet3D,
    # 'UNetwithSEBlock3D': UNetwithSEBlock3D,
    # 'UNetwithSelfattention3D': UNetwithSelfattention3D,
}


class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        """
        :param patience: 如果在多少个epoch内验证集损失没有改善，则提前停止训练
        :param delta: 在认为损失有改善时，损失变化的最小值
        """
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.best_epoch = 0
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_epoch = epoch
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0  # 重置计数器
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Early stopping at epoch {epoch + 1} (no improvement in validation loss for {self.patience} epochs).")
                self.early_stop = True

# 在每次训练之前根据模型名实例化模型
def get_model(model_name):
    return model_dict[model_name]()

def train(model_name, testloader, valloader, epochs, device, earlystoplimit, lr):
    model = get_model(model_name).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    loss_function = MixedMSE(1,0.1)
    early_stopping = EarlyStopping(patience=20, delta=earlystoplimit)
    start_time = time.time()

    best_model = model
    best_val_loss = 10000
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, optimizer, testloader, device, epoch, loss_function)
        scheduler.step()
        val_loss = evaluate(model, valloader, device, loss_function)
        
        # 输出每个epoch的损失
        print(f" Epoch {epoch + 1}: Train Loss: {train_loss:.6f}, Validation Loss: {val_loss:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if epoch > 50 :#设置模型保存间隔
                best_model = model
        early_stopping(val_loss, epoch)
        if early_stopping.early_stop:
            break
    torch.save(best_model.state_dict(), f"/home/linux/3.3lab/outcomes/SSIM_test01_1x/{model_name}.pth")
    training_time = time.time() - start_time
    return {
        'model_name': model_name,
        'model_loss': best_val_loss,
        'training_time': training_time,
    }


def main(args):

    data_transform = {
        "without_jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()]),
        "jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()])}
    # 实例化训练数据集
    data_set = MyDataSet(img_dir=args.img_dir,
                        group_size=10000,
                        size_in = 10000,
                        splition = True,
                        split_shuffle = False,
                        transform=data_transform['without_jet'])
    train_dataset = dataset_2(data_set.train_X, data_set.train_Y)
    val_dataset = dataset_2(data_set.val_X, data_set.val_Y)
    test_dataset = dataset_2(data_set.test_X, data_set.test_Y)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=200, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=200, shuffle=False)
    print(len(train_dataset))
    print(len(test_dataset))
    
    all_results = []
    # 训练每个模型并记录结果
    for model_name in model_dict.keys():
        result = train(model_name, train_dataloader, val_dataloader, epochs=args.epochs,
                                        device=args.device, earlystoplimit=args.earlystoplimit, lr=args.lr)
        all_results.append(result)

    # 输出所有模型的结果
    for result in all_results:
        print(f"Model: {result['model_name']}")
        print(f"Validation Loss: {result['model_loss']}")
        print(f"Training Time: {result['training_time']}")
        print("-" * 50)

class Args:
    def __init__(self):
        self.epochs = 1000
        self.batch_size = 200
        self.lr = 0.001
        self.img_dir = 'Gauss_S1.00_NL0.30_B0.50/Gauss_S1.00_NL0.30_B0.50' 
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.earlystoplimit = 0


opt = Args()

if __name__ == "__main__":
    main(opt)


transformation is not None
8000
1000
 Epoch 1: Train Loss: 0.082360, Validation Loss: 0.099331
 Epoch 2: Train Loss: 0.016473, Validation Loss: 0.011551
 Epoch 3: Train Loss: 0.008303, Validation Loss: 0.007640
 Epoch 4: Train Loss: 0.006425, Validation Loss: 0.005983
 Epoch 5: Train Loss: 0.005379, Validation Loss: 0.005169
 Epoch 6: Train Loss: 0.004763, Validation Loss: 0.004586
 Epoch 7: Train Loss: 0.004388, Validation Loss: 0.004219
 Epoch 8: Train Loss: 0.004070, Validation Loss: 0.003970
 Epoch 9: Train Loss: 0.003853, Validation Loss: 0.003781
 Epoch 10: Train Loss: 0.003739, Validation Loss: 0.003636
 Epoch 11: Train Loss: 0.003505, Validation Loss: 0.003561
 Epoch 12: Train Loss: 0.003428, Validation Loss: 0.004237
 Epoch 13: Train Loss: 0.003297, Validation Loss: 0.003329
 Epoch 14: Train Loss: 0.003128, Validation Loss: 0.003283
 Epoch 15: Train Loss: 0.003031, Validation Loss: 0.003060
 Epoch 16: Train Loss: 0.002947, Validation Loss: 0.002930
 Epoch 17: Train Loss: 0.002

In [3]:
#SSIM 0.5
import os
import random
import numpy as np
import time

import torch
import torch.optim as optim
from torchvision import transforms
import torch.optim.lr_scheduler as lr_scheduler
from model_train2 import CNN, CNNwithSEBlock, CNN3D, CNNwithSEBlock3D, UNet, UNetwithSEBlock, UNetwithSelfattention, UNet3D, UNetwithSEBlock3D, UNetwithSelfattention3D

from DataSet import MaxMinNormalizeGlobalPerChannel,MyDataSet, dataset_2
from train_and_eval import train_one_epoch, evaluate,MixedMSE

random.seed(26)
np.random.seed(26)
torch.manual_seed(26)
torch.cuda.manual_seed(26)
torch.cuda.manual_seed_all(26) 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"  # 或者 ":4096:8"


model_dict = {
    'CNN': CNN,
    'CNNwithSEBlock': CNNwithSEBlock,
    'CNN3D': CNN3D,
    'CNNwithSEBlock3D': CNNwithSEBlock3D,
    # 'UNet': UNet,
    # 'UNetwithSEBlock': UNetwithSEBlock,
    # 'UNetwithSelfattention': UNetwithSelfattention,
    # 'UNet3D': UNet3D,
    # 'UNetwithSEBlock3D': UNetwithSEBlock3D,
    # 'UNetwithSelfattention3D': UNetwithSelfattention3D,
}


class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        """
        :param patience: 如果在多少个epoch内验证集损失没有改善，则提前停止训练
        :param delta: 在认为损失有改善时，损失变化的最小值
        """
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.best_epoch = 0
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_epoch = epoch
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0  # 重置计数器
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Early stopping at epoch {epoch + 1} (no improvement in validation loss for {self.patience} epochs).")
                self.early_stop = True

# 在每次训练之前根据模型名实例化模型
def get_model(model_name):
    return model_dict[model_name]()

def train(model_name, testloader, valloader, epochs, device, earlystoplimit, lr):
    model = get_model(model_name).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    loss_function = MixedMSE(1,0.5)
    early_stopping = EarlyStopping(patience=20, delta=earlystoplimit)
    start_time = time.time()

    best_model = model
    best_val_loss = 10000
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, optimizer, testloader, device, epoch, loss_function)
        scheduler.step()
        val_loss = evaluate(model, valloader, device, loss_function)
        
        # 输出每个epoch的损失
        print(f" Epoch {epoch + 1}: Train Loss: {train_loss:.6f}, Validation Loss: {val_loss:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if epoch > 50 :#设置模型保存间隔
                best_model = model
        early_stopping(val_loss, epoch)
        if early_stopping.early_stop:
            break
    torch.save(best_model.state_dict(), f"/home/linux/3.3lab/outcomes/SSIM_test01_5x/{model_name}.pth")
    training_time = time.time() - start_time
    return {
        'model_name': model_name,
        'model_loss': best_val_loss,
        'training_time': training_time,
    }


def main(args):

    data_transform = {
        "without_jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()]),
        "jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()])}
    # 实例化训练数据集
    data_set = MyDataSet(img_dir=args.img_dir,
                        group_size=10000,
                        size_in = 10000,
                        splition = True,
                        split_shuffle = False,
                        transform=data_transform['without_jet'])
    train_dataset = dataset_2(data_set.train_X, data_set.train_Y)
    val_dataset = dataset_2(data_set.val_X, data_set.val_Y)
    test_dataset = dataset_2(data_set.test_X, data_set.test_Y)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=200, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=200, shuffle=False)
    print(len(train_dataset))
    print(len(test_dataset))
    
    all_results = []
    # 训练每个模型并记录结果
    for model_name in model_dict.keys():
        result = train(model_name, train_dataloader, val_dataloader, epochs=args.epochs,
                                        device=args.device, earlystoplimit=args.earlystoplimit, lr=args.lr)
        all_results.append(result)

    # 输出所有模型的结果
    for result in all_results:
        print(f"Model: {result['model_name']}")
        print(f"Validation Loss: {result['model_loss']}")
        print(f"Training Time: {result['training_time']}")
        print("-" * 50)

class Args:
    def __init__(self):
        self.epochs = 1000
        self.batch_size = 200
        self.lr = 0.001
        self.img_dir = 'Gauss_S1.00_NL0.30_B0.50/Gauss_S1.00_NL0.30_B0.50' 
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.earlystoplimit = 0


opt = Args()

if __name__ == "__main__":
    main(opt)


transformation is not None
8000
1000
 Epoch 1: Train Loss: 0.134431, Validation Loss: 0.390555
 Epoch 2: Train Loss: 0.034158, Validation Loss: 0.028937
 Epoch 3: Train Loss: 0.024738, Validation Loss: 0.025623
 Epoch 4: Train Loss: 0.020408, Validation Loss: 0.020385
 Epoch 5: Train Loss: 0.018908, Validation Loss: 0.018437
 Epoch 6: Train Loss: 0.016665, Validation Loss: 0.016298
 Epoch 7: Train Loss: 0.015578, Validation Loss: 0.015241
 Epoch 8: Train Loss: 0.014633, Validation Loss: 0.014485
 Epoch 9: Train Loss: 0.014047, Validation Loss: 0.013915
 Epoch 10: Train Loss: 0.013289, Validation Loss: 0.013676
 Epoch 11: Train Loss: 0.012902, Validation Loss: 0.014238
 Epoch 12: Train Loss: 0.012255, Validation Loss: 0.012054
 Epoch 13: Train Loss: 0.011790, Validation Loss: 0.011790
 Epoch 14: Train Loss: 0.011371, Validation Loss: 0.011796
 Epoch 15: Train Loss: 0.010866, Validation Loss: 0.011627
 Epoch 16: Train Loss: 0.010821, Validation Loss: 0.012710
 Epoch 17: Train Loss: 0.010

In [4]:
#SSIM 0.05
import os
import random
import numpy as np
import time

import torch
import torch.optim as optim
from torchvision import transforms
import torch.optim.lr_scheduler as lr_scheduler
from model_train2 import CNN, CNNwithSEBlock, CNN3D, CNNwithSEBlock3D, UNet, UNetwithSEBlock, UNetwithSelfattention, UNet3D, UNetwithSEBlock3D, UNetwithSelfattention3D

from DataSet import MaxMinNormalizeGlobalPerChannel,MyDataSet, dataset_2
from train_and_eval import train_one_epoch, evaluate,MixedMSE

random.seed(26)
np.random.seed(26)
torch.manual_seed(26)
torch.cuda.manual_seed(26)
torch.cuda.manual_seed_all(26) 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"  # 或者 ":4096:8"


model_dict = {
    'CNN': CNN,
    'CNNwithSEBlock': CNNwithSEBlock,
    'CNN3D': CNN3D,
    'CNNwithSEBlock3D': CNNwithSEBlock3D,
    # 'UNet': UNet,
    # 'UNetwithSEBlock': UNetwithSEBlock,
    # 'UNetwithSelfattention': UNetwithSelfattention,
    # 'UNet3D': UNet3D,
    # 'UNetwithSEBlock3D': UNetwithSEBlock3D,
    # 'UNetwithSelfattention3D': UNetwithSelfattention3D,
}


class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        """
        :param patience: 如果在多少个epoch内验证集损失没有改善，则提前停止训练
        :param delta: 在认为损失有改善时，损失变化的最小值
        """
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.best_epoch = 0
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_epoch = epoch
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0  # 重置计数器
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Early stopping at epoch {epoch + 1} (no improvement in validation loss for {self.patience} epochs).")
                self.early_stop = True

# 在每次训练之前根据模型名实例化模型
def get_model(model_name):
    return model_dict[model_name]()

def train(model_name, testloader, valloader, epochs, device, earlystoplimit, lr):
    model = get_model(model_name).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    loss_function = MixedMSE(1,0.05)
    early_stopping = EarlyStopping(patience=20, delta=earlystoplimit)
    start_time = time.time()

    best_model = model
    best_val_loss = 10000
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, optimizer, testloader, device, epoch, loss_function)
        scheduler.step()
        val_loss = evaluate(model, valloader, device, loss_function)
        
        # 输出每个epoch的损失
        print(f" Epoch {epoch + 1}: Train Loss: {train_loss:.6f}, Validation Loss: {val_loss:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if epoch > 50 :#设置模型保存间隔
                best_model = model
        early_stopping(val_loss, epoch)
        if early_stopping.early_stop:
            break
    torch.save(best_model.state_dict(), f"/home/linux/3.3lab/outcomes/SSIM_test01_0.5x/{model_name}.pth")
    training_time = time.time() - start_time
    return {
        'model_name': model_name,
        'model_loss': best_val_loss,
        'training_time': training_time,
    }


def main(args):

    data_transform = {
        "without_jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()]),
        "jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()])}
    # 实例化训练数据集
    data_set = MyDataSet(img_dir=args.img_dir,
                        group_size=10000,
                        size_in = 10000,
                        splition = True,
                        split_shuffle = False,
                        transform=data_transform['without_jet'])
    train_dataset = dataset_2(data_set.train_X, data_set.train_Y)
    val_dataset = dataset_2(data_set.val_X, data_set.val_Y)
    test_dataset = dataset_2(data_set.test_X, data_set.test_Y)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=200, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=200, shuffle=False)
    print(len(train_dataset))
    print(len(test_dataset))
    
    all_results = []
    # 训练每个模型并记录结果
    for model_name in model_dict.keys():
        result = train(model_name, train_dataloader, val_dataloader, epochs=args.epochs,
                                        device=args.device, earlystoplimit=args.earlystoplimit, lr=args.lr)
        all_results.append(result)

    # 输出所有模型的结果
    for result in all_results:
        print(f"Model: {result['model_name']}")
        print(f"Validation Loss: {result['model_loss']}")
        print(f"Training Time: {result['training_time']}")
        print("-" * 50)

class Args:
    def __init__(self):
        self.epochs = 1000
        self.batch_size = 200
        self.lr = 0.001
        self.img_dir = 'Gauss_S1.00_NL0.30_B0.50/Gauss_S1.00_NL0.30_B0.50' 
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.earlystoplimit = 0


opt = Args()

if __name__ == "__main__":
    main(opt)


transformation is not None
8000
1000
 Epoch 1: Train Loss: 0.054032, Validation Loss: 0.045697
 Epoch 2: Train Loss: 0.011120, Validation Loss: 0.010547
 Epoch 3: Train Loss: 0.005477, Validation Loss: 0.004876
 Epoch 4: Train Loss: 0.004210, Validation Loss: 0.003916
 Epoch 5: Train Loss: 0.003536, Validation Loss: 0.003373
 Epoch 6: Train Loss: 0.003106, Validation Loss: 0.003050
 Epoch 7: Train Loss: 0.002823, Validation Loss: 0.002743
 Epoch 8: Train Loss: 0.002626, Validation Loss: 0.002572
 Epoch 9: Train Loss: 0.002471, Validation Loss: 0.002420
 Epoch 10: Train Loss: 0.002351, Validation Loss: 0.002371
 Epoch 11: Train Loss: 0.002249, Validation Loss: 0.002232
 Epoch 12: Train Loss: 0.002162, Validation Loss: 0.002144
 Epoch 13: Train Loss: 0.002097, Validation Loss: 0.002501
 Epoch 14: Train Loss: 0.002005, Validation Loss: 0.002060
 Epoch 15: Train Loss: 0.001940, Validation Loss: 0.001933
 Epoch 16: Train Loss: 0.001888, Validation Loss: 0.001935
 Epoch 17: Train Loss: 0.001

In [5]:
#SSIM 0
import os
import random
import numpy as np
import time

import torch
import torch.optim as optim
from torchvision import transforms
import torch.optim.lr_scheduler as lr_scheduler
from model_train2 import CNN, CNNwithSEBlock, CNN3D, CNNwithSEBlock3D, UNet, UNetwithSEBlock, UNetwithSelfattention, UNet3D, UNetwithSEBlock3D, UNetwithSelfattention3D

from DataSet import MaxMinNormalizeGlobalPerChannel,MyDataSet, dataset_2
from train_and_eval import train_one_epoch, evaluate,MixedMSE

random.seed(26)
np.random.seed(26)
torch.manual_seed(26)
torch.cuda.manual_seed(26)
torch.cuda.manual_seed_all(26) 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"  # 或者 ":4096:8"


model_dict = {
    'CNN': CNN,
    'CNNwithSEBlock': CNNwithSEBlock,
    'CNN3D': CNN3D,
    'CNNwithSEBlock3D': CNNwithSEBlock3D,
    # 'UNet': UNet,
    # 'UNetwithSEBlock': UNetwithSEBlock,
    # 'UNetwithSelfattention': UNetwithSelfattention,
    # 'UNet3D': UNet3D,
    # 'UNetwithSEBlock3D': UNetwithSEBlock3D,
    # 'UNetwithSelfattention3D': UNetwithSelfattention3D,
}


class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        """
        :param patience: 如果在多少个epoch内验证集损失没有改善，则提前停止训练
        :param delta: 在认为损失有改善时，损失变化的最小值
        """
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.best_epoch = 0
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_epoch = epoch
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0  # 重置计数器
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Early stopping at epoch {epoch + 1} (no improvement in validation loss for {self.patience} epochs).")
                self.early_stop = True

# 在每次训练之前根据模型名实例化模型
def get_model(model_name):
    return model_dict[model_name]()

def train(model_name, testloader, valloader, epochs, device, earlystoplimit, lr):
    model = get_model(model_name).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    loss_function = MixedMSE(1,0)
    early_stopping = EarlyStopping(patience=20, delta=earlystoplimit)
    start_time = time.time()

    best_model = model
    best_val_loss = 10000
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, optimizer, testloader, device, epoch, loss_function)
        scheduler.step()
        val_loss = evaluate(model, valloader, device, loss_function)
        
        # 输出每个epoch的损失
        print(f" Epoch {epoch + 1}: Train Loss: {train_loss:.6f}, Validation Loss: {val_loss:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if epoch > 50 :#设置模型保存间隔
                best_model = model
        early_stopping(val_loss, epoch)
        if early_stopping.early_stop:
            break
    torch.save(best_model.state_dict(), f"/home/linux/3.3lab/outcomes/SSIM_comparison/{model_name}.pth")
    training_time = time.time() - start_time
    return {
        'model_name': model_name,
        'model_loss': best_val_loss,
        'training_time': training_time,
    }


def main(args):

    data_transform = {
        "without_jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()]),
        "jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()])}
    # 实例化训练数据集
    data_set = MyDataSet(img_dir=args.img_dir,
                        group_size=10000,
                        size_in = 10000,
                        splition = True,
                        split_shuffle = False,
                        transform=data_transform['without_jet'])
    train_dataset = dataset_2(data_set.train_X, data_set.train_Y)
    val_dataset = dataset_2(data_set.val_X, data_set.val_Y)
    test_dataset = dataset_2(data_set.test_X, data_set.test_Y)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=200, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=200, shuffle=False)
    print(len(train_dataset))
    print(len(test_dataset))
    
    all_results = []
    # 训练每个模型并记录结果
    for model_name in model_dict.keys():
        result = train(model_name, train_dataloader, val_dataloader, epochs=args.epochs,
                                        device=args.device, earlystoplimit=args.earlystoplimit, lr=args.lr)
        all_results.append(result)

    # 输出所有模型的结果
    for result in all_results:
        print(f"Model: {result['model_name']}")
        print(f"Validation Loss: {result['model_loss']}")
        print(f"Training Time: {result['training_time']}")
        print("-" * 50)

class Args:
    def __init__(self):
        self.epochs = 1000
        self.batch_size = 200
        self.lr = 0.001
        self.img_dir = 'Gauss_S1.00_NL0.30_B0.50/Gauss_S1.00_NL0.30_B0.50' 
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.earlystoplimit = 0


opt = Args()

if __name__ == "__main__":
    main(opt)


transformation is not None
8000
1000
 Epoch 1: Train Loss: 0.020664, Validation Loss: 0.029594
 Epoch 2: Train Loss: 0.003047, Validation Loss: 0.003182
 Epoch 3: Train Loss: 0.001342, Validation Loss: 0.001283
 Epoch 4: Train Loss: 0.000975, Validation Loss: 0.000897
 Epoch 5: Train Loss: 0.000837, Validation Loss: 0.000789
 Epoch 6: Train Loss: 0.000753, Validation Loss: 0.000721
 Epoch 7: Train Loss: 0.000693, Validation Loss: 0.000671
 Epoch 8: Train Loss: 0.000649, Validation Loss: 0.000629
 Epoch 9: Train Loss: 0.000612, Validation Loss: 0.000600
 Epoch 10: Train Loss: 0.000583, Validation Loss: 0.000575
 Epoch 11: Train Loss: 0.000557, Validation Loss: 0.000554
 Epoch 12: Train Loss: 0.000537, Validation Loss: 0.000534
 Epoch 13: Train Loss: 0.000519, Validation Loss: 0.000526
 Epoch 14: Train Loss: 0.000506, Validation Loss: 0.000506
 Epoch 15: Train Loss: 0.000490, Validation Loss: 0.000493
 Epoch 16: Train Loss: 0.000477, Validation Loss: 0.000480
 Epoch 17: Train Loss: 0.000

In [7]:
#SSIM all
import os
import random
import numpy as np
import time

import torch
import torch.optim as optim
from torchvision import transforms
import torch.optim.lr_scheduler as lr_scheduler
from model_train2 import CNN, CNNwithSEBlock, CNN3D, CNNwithSEBlock3D, UNet, UNetwithSEBlock, UNetwithSelfattention, UNet3D, UNetwithSEBlock3D, UNetwithSelfattention3D

from DataSet import MaxMinNormalizeGlobalPerChannel,MyDataSet, dataset_2
from train_and_eval import train_one_epoch, evaluate,MixedMSE

random.seed(26)
np.random.seed(26)
torch.manual_seed(26)
torch.cuda.manual_seed(26)
torch.cuda.manual_seed_all(26) 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"  # 或者 ":4096:8"


model_dict = {
    'CNN': CNN,
    'CNNwithSEBlock': CNNwithSEBlock,
    'CNN3D': CNN3D,
    'CNNwithSEBlock3D': CNNwithSEBlock3D,
    # 'UNet': UNet,
    # 'UNetwithSEBlock': UNetwithSEBlock,
    # 'UNetwithSelfattention': UNetwithSelfattention,
    # 'UNet3D': UNet3D,
    # 'UNetwithSEBlock3D': UNetwithSEBlock3D,
    # 'UNetwithSelfattention3D': UNetwithSelfattention3D,
}


class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        """
        :param patience: 如果在多少个epoch内验证集损失没有改善，则提前停止训练
        :param delta: 在认为损失有改善时，损失变化的最小值
        """
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.best_epoch = 0
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_epoch = epoch
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0  # 重置计数器
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Early stopping at epoch {epoch + 1} (no improvement in validation loss for {self.patience} epochs).")
                self.early_stop = True

# 在每次训练之前根据模型名实例化模型
def get_model(model_name):
    return model_dict[model_name]()

def train(model_name, testloader, valloader, epochs, device, earlystoplimit, lr):
    model = get_model(model_name).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    loss_function = MixedMSE(0,0.1)
    early_stopping = EarlyStopping(patience=20, delta=earlystoplimit)
    start_time = time.time()

    best_model = model
    best_val_loss = 10000
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, optimizer, testloader, device, epoch, loss_function)
        scheduler.step()
        val_loss = evaluate(model, valloader, device, loss_function)
        
        # 输出每个epoch的损失
        print(f" Epoch {epoch + 1}: Train Loss: {train_loss:.6f}, Validation Loss: {val_loss:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if epoch > 50 :#设置模型保存间隔
                best_model = model
        early_stopping(val_loss, epoch)
        if early_stopping.early_stop:
            break
    torch.save(best_model.state_dict(), f"/home/linux/3.3lab/outcomes/SSIM_comparison2/{model_name}.pth")
    training_time = time.time() - start_time
    return {
        'model_name': model_name,
        'model_loss': best_val_loss,
        'training_time': training_time,
    }


def main(args):

    data_transform = {
        "without_jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()]),
        "jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()])}
    # 实例化训练数据集
    data_set = MyDataSet(img_dir=args.img_dir,
                        group_size=10000,
                        size_in = 10000,
                        splition = True,
                        split_shuffle = False,
                        transform=data_transform['without_jet'])
    train_dataset = dataset_2(data_set.train_X, data_set.train_Y)
    val_dataset = dataset_2(data_set.val_X, data_set.val_Y)
    test_dataset = dataset_2(data_set.test_X, data_set.test_Y)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=200, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=200, shuffle=False)
    print(len(train_dataset))
    print(len(test_dataset))
    
    all_results = []
    # 训练每个模型并记录结果
    for model_name in model_dict.keys():
        result = train(model_name, train_dataloader, val_dataloader, epochs=args.epochs,
                                        device=args.device, earlystoplimit=args.earlystoplimit, lr=args.lr)
        all_results.append(result)

    # 输出所有模型的结果
    for result in all_results:
        print(f"Model: {result['model_name']}")
        print(f"Validation Loss: {result['model_loss']}")
        print(f"Training Time: {result['training_time']}")
        print("-" * 50)

class Args:
    def __init__(self):
        self.epochs = 1000
        self.batch_size = 200
        self.lr = 0.001
        self.img_dir = 'Gauss_S1.00_NL0.30_B0.50/Gauss_S1.00_NL0.30_B0.50' 
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.earlystoplimit = 0


opt = Args()

if __name__ == "__main__":
    main(opt)


transformation is not None
8000
1000
 Epoch 1: Train Loss: 0.025253, Validation Loss: 0.064641
 Epoch 2: Train Loss: 0.006185, Validation Loss: 0.005309
 Epoch 3: Train Loss: 0.004491, Validation Loss: 0.005149
 Epoch 4: Train Loss: 0.003757, Validation Loss: 0.003665
 Epoch 5: Train Loss: 0.003356, Validation Loss: 0.003276
 Epoch 6: Train Loss: 0.003086, Validation Loss: 0.003024
 Epoch 7: Train Loss: 0.002853, Validation Loss: 0.002780
 Epoch 8: Train Loss: 0.002703, Validation Loss: 0.003007
 Epoch 9: Train Loss: 0.002588, Validation Loss: 0.002508
 Epoch 10: Train Loss: 0.002439, Validation Loss: 0.002667
 Epoch 11: Train Loss: 0.002349, Validation Loss: 0.002625
 Epoch 12: Train Loss: 0.002228, Validation Loss: 0.002218
 Epoch 13: Train Loss: 0.002148, Validation Loss: 0.002196
 Epoch 14: Train Loss: 0.002065, Validation Loss: 0.002116
 Epoch 15: Train Loss: 0.002004, Validation Loss: 0.002167
 Epoch 16: Train Loss: 0.001943, Validation Loss: 0.001993
 Epoch 17: Train Loss: 0.001

In [ ]:
import os
import random
import numpy as np
import time

import torch
import torch.optim as optim
from torchvision import transforms
import torch.optim.lr_scheduler as lr_scheduler
from model_train import CNN, CNNwithSEBlock, CNN3D, CNNwithSEBlock3D, UNet, UNetwithSEBlock, UNetwithSelfattention, UNet3D, UNetwithSEBlock3D, UNetwithSelfattention3D

from DataSet_2 import MaxMinNormalizeGlobalPerChannel,MyDataSet, dataset_2
from train_and_eval import train_one_epoch, evaluate, MixedMSE

random.seed(26)
np.random.seed(26)
torch.manual_seed(26)
torch.cuda.manual_seed(26)
torch.cuda.manual_seed_all(26) 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"  # 或者 ":4096:8"


model_dict = {
    'CNN': CNN,
    # 'CNNwithSEBlock': CNNwithSEBlock,
    # 'CNN3D': CNN3D,
    # 'CNNwithSEBlock3D': CNNwithSEBlock3D,
    # 'UNet': UNet,
    # 'UNetwithSEBlock': UNetwithSEBlock,
    # 'UNetwithSelfattention': UNetwithSelfattention,
    # 'UNet3D': UNet3D,
    # 'UNetwithSEBlock3D': UNetwithSEBlock3D,
    # 'UNetwithSelfattention3D': UNetwithSelfattention3D,
}


class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        """
        :param patience: 如果在多少个epoch内验证集损失没有改善，则提前停止训练
        :param delta: 在认为损失有改善时，损失变化的最小值
        """
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.best_epoch = 0
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_epoch = epoch
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0  # 重置计数器
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Early stopping at epoch {epoch + 1} (no improvement in validation loss for {self.patience} epochs).")
                self.early_stop = True

# 在每次训练之前根据模型名实例化模型
def get_model(model_name):
    return model_dict[model_name]()

def train(model_name, testloader, valloader, epochs, device, earlystoplimit, lr):
    model = get_model(model_name).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    loss_function = MixedMSE(1,0.05)
    early_stopping = EarlyStopping(patience=20, delta=earlystoplimit)
    start_time = time.time()

    best_model = model
    best_val_loss = 10000
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, optimizer, testloader, device, epoch, loss_function)
        scheduler.step()
        val_loss = evaluate(model, valloader, device, loss_function)
        
        # 输出每个epoch的损失
        print(f" Epoch {epoch + 1}: Train Loss: {train_loss:.6f}, Validation Loss: {val_loss:.6f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if epoch > 50 :#设置模型保存间隔
                best_model = model
        early_stopping(val_loss, epoch)
        if early_stopping.early_stop:
            break
    torch.save(best_model.state_dict(), f"/home/linux/3.3lab/outcomes/new_minmax/{model_name}.pth")
    training_time = time.time() - start_time
    return {
        'model_name': model_name,
        'model_loss': best_val_loss,
        'training_time': training_time,
    }


def main(args):

    data_transform = {
        "without_jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()]),
        "jet": transforms.Compose([MaxMinNormalizeGlobalPerChannel()])}
    # 实例化训练数据集
    data_set = MyDataSet(img_dir=args.img_dir,
                        group_size=10000,
                        size_in = 10000,
                        splition = True,
                        split_shuffle = False,
                        transform=data_transform['without_jet'])
    train_dataset = dataset_2(data_set.train_X, data_set.train_Y)
    val_dataset = dataset_2(data_set.val_X, data_set.val_Y)
    test_dataset = dataset_2(data_set.test_X, data_set.test_Y)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=200, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=200, shuffle=False)
    print(len(train_dataset))
    print(len(test_dataset))
    
    all_results = []
    # 训练每个模型并记录结果
    for model_name in model_dict.keys():
        result = train(model_name, train_dataloader, val_dataloader, epochs=args.epochs,
                                        device=args.device, earlystoplimit=args.earlystoplimit, lr=args.lr)
        all_results.append(result)

    # 输出所有模型的结果
    for result in all_results:
        print(f"Model: {result['model_name']}")
        print(f"Validation Loss: {result['model_loss']}")
        print(f"Training Time: {result['training_time']}")
        print("-" * 50)

class Args:
    def __init__(self):
        self.epochs = 1000
        self.batch_size = 200
        self.lr = 0.001
        self.img_dir = 'Gauss_S1.00_NL0.30_B0.50/Gauss_S1.00_NL0.30_B0.50' 
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.earlystoplimit = 0


opt = Args()

if __name__ == "__main__":
    main(opt)

transformation is not None
Min: 0.0
Max: 17.789060592651367
Global Min: tensor([[[[0.]]]])
Global Max: tensor([[[[17.7891]]]])
8000
1000
 Epoch 1: Train Loss: 0.048071, Validation Loss: 0.044820
 Epoch 2: Train Loss: 0.010151, Validation Loss: 0.008157
